In [19]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py transformers torch einops pypdf onnxruntime openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import os 
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.environ['LANGCHAIN_TRACING_V2'] = 'true'
LANGCHAIN_ENDPOINT = os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
FIRECRAWL_API_KEY = os.environ['FIRECRAWL_API_KEY']

In [9]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3.1:latest", format="json", temperature=0)

/var/folders/86/7vl_g9y936j10tb2scb6w_jr0000gn/T/ipykernel_14860/3253887872.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:latest", format="json", temperature=0)


In [21]:
from langchain_community.document_loaders import PyPDFLoader

docs = ('/Users/tt-josue-sanchez/Downloads/FY25 NSI Preliminary Package.pdf')

loader = PyPDFLoader(docs)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print(f"{pages[0].metadata}\n")
print(pages[0].page_content)


{'source': '/Users/tt-josue-sanchez/Downloads/FY25 NSI Preliminary Package.pdf', 'page': 0}

 
 1 of 6 FY-25 
 
 
 NAVSEA 
 STANDARD ITEM 
 NUMERICAL INDEX 
 
 
 FY-25    
 
ITEM NO. 
 
 
TITLE 
UTILIZATIO
N 
 CATEGORY   
  
DATE 
009-01 General Criteria; accomplish I  xx OCT 2023 
009-02 Environmental Compliance Report for 
Material Usage; accomplish 
I  25 OCT 2022 
009-03 Toxic and Hazardous Substance; control I  25 OCT 2022 
009-04 Quality Management System; provide I  xx OCT 2023 
009-05 Temporary Access; accomplish I  25 OCT 2022 
009-06 Maintaining Protection and Cleanliness 
from Non-Radioactive Operation; 
accomplish 
I  xx OCT 2023 
009-07 Confined Space Entry, and Certification; 
accomplish 
I  xx OCT 2023 
009-08 Shipboard Fire Protection and Fire 
Prevention; accomplish 
I  xx OCT 2023 
009-09 Process Control Procedure (PCP); provide 
and accomplish 
II  25 OCT 2022 
009-10 Asbestos-Containing Material (ACM); 
control                   
I  25 OCT 2022 
009-11 Insulation an

In [22]:
from transformers import AutoModel, AutoTokenizer
import torch
from langchain_community.vectorstores import Chroma

# Load the embedding model and tokenizer
model_name = "nomic-ai/nomic-embed-text-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to generate embeddings using the loaded model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].cpu().numpy().tolist()  # Convert ndarray to list

# Wrapper class to use with Chroma
class CustomEmbedding:
    def embed_documents(self, texts):
        return [embed_text(text) for text in texts]

    def embed_query(self, text):
        return embed_text(text)

# Instantiate the embedding class
custom_embedding = CustomEmbedding()

# Add documents with embeddings to the vectorDB using the embedding class
vectorstore = Chroma.from_documents(
    documents=pages,
    collection_name="rag-chroma",
    embedding=custom_embedding,  # Use the embedding class instance
)

retriever = vectorstore.as_retriever()

/Users/tt-josue-sanchez/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/eb02ceb48c1fdcc477ff1925c9732c379f0f0d1f/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issue

In [23]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assitant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Return the text with the information in bullet points.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["question", "document"]
)

#Post processing
def format_doc(docs):
    return"\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

#Run

question = "what are the requirements of 009-04?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

{ 
  "009-04": [
    "Comply with security requirements.",
    "Verify that personnel, including subcontractor's personnel, are cleared for the required level of security classification for handling, repair, installation, and testing of classified equipment and for access to areas of the ship which require a specific security clearance."
  ]
}


In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
import pandas as pd

# Define the prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Return the text with the information in bullet points.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["question", "document"]
)

# Post-processing function
def format_doc(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the chain
rag_chain = prompt | llm | StrOutputParser()

# Run the pipeline
question = "What is the information found regarding section 3?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})

# Convert the generation result into an Excel file
output_file = "generation_output.xlsx"
data = {"Question": [question], "Generated Answer": [generation]}
df = pd.DataFrame(data)

# Save to Excel
df.to_excel(output_file, index=False)
print(f"The generation has been saved to {output_file}.")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


The generation has been saved to generation_output.xlsx.
